In [ ]:
import os
import pandas as pd
import numpy as np
import random
from pandas import DataFrame
from datetime import datetime


In [ ]:
# < Important note > [ Please download input data from https://mega.nz/#F!CeYGDKyS!uqkmWJ4E2XSGJp_C2VO2gg]
# IC50evaluation//Dataset//Scenario6_MDC-9K_minmax
workdir = "path"
dataset = np.load(workdir + "//MDC9k_190510_ccle_cell_gdsc_mut_drug_info.npz") # input file
ss0 = np.load(workdir + '//MDC9k_190510_ccle_cell_gdsc_mut_drug_info_r0_9_1.npz') # split for training and test

In [ ]:
x = dataset['x']
y = dataset['y']
# y_linear = dataset['y_lnIC50']
ss0_train = ss0['train']
ss0_test = ss0['test']

In [ ]:
training_image_array, training_label_array = x[ss0_train], y[ss0_train]
test_image_array, test_label_array = x[ss0_test], y[ss0_test]

# # In[9]:
# ori = training_image_array
# bat = np.zeros((ori.shape[0],178))
# cat = np.hstack([ori,bat])
# training_image_array = cat

# # In[8]:
# training_image_array.shape

# # In[10]:
# ori2 = test_image_array
# bat2 = np.zeros((ori2.shape[0],178))
# cat2 = np.hstack([ori2,bat2])
# test_image_array = cat2


In [ ]:
# In[15]:
ab =[]
for i in range(100,200):
    ab.append(len(training_image_array) % i)
    
print(min(ab), ab.index(min(ab)))

In [ ]:
x.shape

In [ ]:
training_image_array.shape

In [ ]:
train_X, train_y, test_X, test_y = training_image_array, training_label_array, test_image_array, test_label_array

# if K.image_data_format() == 'channels_first':
#     train_X = train_X.reshape(train_X.shape[0], 1, img_rows, img_cols)
#     test_X = test_X.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     train_X = train_X.reshape(train_X.shape[0], img_rows, img_cols, 1)
#     test_X = test_X.reshape(test_X.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)


In [ ]:
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
train_X = np.nan_to_num(train_X)
test_X = np.nan_to_num(test_X)

In [ ]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
# train_X /= 255
# test_X /= 255
print('train_X shape:', train_X.shape)
print(train_X.shape[0], 'train samples')
print(test_X.shape[0], 'test samples')

In [ ]:
# Model fitting
StartTime8 = datetime.now()
print("StartTime :", StartTime8)
from sklearn.linear_model import Ridge

ridge_regressor = Ridge(alpha=0.001)

ridge_regressor.fit(train_X,train_y)

EndTime8 = datetime.now()
print("EndTime :", EndTime8)

In [ ]:
import matplotlib
from matplotlib import pyplot as plt

In [ ]:
predicted_classes = ridge_regressor.predict(test_X)

In [ ]:
predicted_value = predicted_classes

In [ ]:
# Save model
import pickle
from sklearn.externals import joblib
workdir = "D://DrugResponsivenessGuideline_log//Result//MDC-9K//Ridge"
joblib.dump(ridge_regressor, workdir + '//MDC_9K_minmax_Ridge_model.pkl') 


In [ ]:
# model loading test
clf_from_joblib = joblib.load(workdir + '//MDC_9K_minmax_Ridge_model.pkl') 
clf_from_joblib.predict(test_X)

In [ ]:
a = pd.DataFrame(predicted_value)
b = pd.DataFrame(test_label_array)
c = pd.concat([a,b], axis=1)
c.columns=["Predicted","Test"]

In [ ]:
workdir = "E://Ronny_TF//Ronny_IC50_RMSE//method_comparison//190513_ridge_regression_w_ccle_gdsc_mut_drug"
c.to_csv(workdir + '//190513_ridge_regression_w_ccle_gdsc_mut_drug_result.csv')


In [ ]:
predicted_value.shape

In [ ]:
from scipy.stats import linregress
linregress(b[0], a[0])

In [ ]:
# scatter plot
plt.scatter(test_label_array,predicted_value,c='red')
plt.xlabel('test_IC50_value')
plt.ylabel('predicted_IC50_value')
plt.savefig(workdir + '//190513_ridge_regression_w_ccle_gdsc_mut_drug_test_scatterplot.png')

In [ ]:
# RMSE
rse = ((b[0]-a[0])**2).sum()
mse = rse / len(b)
print("Final rmse value is =",np.sqrt(mse))

In [ ]:
# R-squared value

from sklearn.metrics import r2_score
r2_value = r2_score(b, a) 
print(r2_value)